In [374]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt

dataset = load_dataset('lukebarousse/data_jobs')


In [375]:
df = dataset['train'].to_pandas()
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df.head(3)

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."


In [ ]:
df_cleaned = df.dropna(subset = ['salary_year_avg'])
print(f'the original df rows are {len(df)}')
print(f'the filtered df rows are {len(df_cleaned)}')
print(f'the rows removed are {len(df)-len(df_cleaned)}')

In [ ]:
df_cleaned = df.drop_duplicates(subset = ['job_location'])
print(f'the original # of rows are {len(df)}')
print(f'the cleaned # of rows are {len(df_cleaned)}')
print(f'the rows removed are {len(df)-len(df_cleaned)}')

In [ ]:
salary_before = df['salary_rate'].head(10)

df['salary_rate'] = df['salary_rate'].fillna(value = 'Unknown')
salary_after = df['salary_rate'].head(10)

salary_before, salary_after

In [ ]:
df_copy = df.copy()
# print(id(df_copy),id(df))
df_copy.dropna(subset = ['salary_year_avg'],inplace=True)
df_copy1 = df_copy.dropna(subset = ['salary_year_avg'])

type(df_copy1), df_copy1.head()
# df_copy.info()


In [ ]:
df_frac = df.sample(frac=0.1)
df_frac.head()

In [ ]:
df_copy = df.copy()
median = df_copy['salary_year_avg'].median()
df_copy['salary_year_avg'] = df_copy['salary_year_avg'].fillna(value=median)
df_copy.head()

In [ ]:
df.pivot_table(index=['job_title_short','job_country'],aggfunc='size').head(10)
df.head()

In [ ]:
df_new = df[df['job_title_short']=='Data Scientist'].copy()
df_pivot = df_new.pivot_table(values='salary_year_avg',index=['company_name','job_country'],aggfunc='median')

df_pivot_filter = df_pivot[df_pivot['salary_year_avg'] > 200000]
df_pivot_filter.head(10)

In [228]:
df_copy = df.copy()
df_copy.set_index('job_posted_date',inplace=True)

In [ ]:
df_copy.reset_index(drop=True,inplace=True)
# df_copy.drop(labels='job_posted_date',axis=1, inplace=True)
df_copy.head()

In [ ]:
df_copy = df.copy()
df_pivot = df_copy.pivot_table(values='salary_year_avg',index='company_name',aggfunc='median')
df_pivot
df_pivot.sort_index(ascending=False,inplace=True)
df_pivot.head()

In [ ]:
df_copy = df.copy()
df_pivot = df_copy.pivot_table(values='salary_year_avg',index='job_title_short',aggfunc='median')
df_pivot.reset_index(inplace=True)
df_pivot
df_pivot.set_index('salary_year_avg',inplace=True)
df_pivot.head()

In [260]:
## Skill count per month for data analysts
df_US = df[df['job_country'] == 'United States'].copy()

# extract the month name from 'job_posted_date'
df_US['job_posted_month'] = df_US['job_posted_date'].dt.strftime('%B')
# df_US['job_posted_month']

df_US_pivot = df_US.pivot_table(index='job_posted_month', columns='job_title_short', aggfunc='size')
# df_US_pivot


In [ ]:
# sort job_posted_month_name in df_US_pivot by job_posted_month value
df_US_pivot = df_US_pivot.reset_index()
df_US_pivot['job_posted_month_no'] = pd.to_datetime(df_US_pivot['job_posted_month'], format='%B').dt.month
df_US_pivot = df_US_pivot.sort_values('job_posted_month_no')
df_US_pivot = df_US_pivot.set_index('job_posted_month')
df_US_pivot = df_US_pivot.drop(columns='job_posted_month_no')

df_US_pivot

In [265]:
top_3 = df_US['job_title_short'].value_counts().head(3)
top_3 = top_3.index.tolist()
top_3

['Data Analyst', 'Data Scientist', 'Data Engineer']

In [ ]:

df_us_jobs = df[df['job_country']=='United States'].copy()
df_us_jobs['job_posted_month'] = df_us_jobs['job_posted_date'].dt.to_period('M')
df_monthly_postings = df_us_jobs.groupby('job_posted_month').size().reset_index(name='postings_count')
merge_df = pd.merge(df_us_jobs,df_monthly_postings, how='right',on='job_posted_month')
# df_us_jobs
df_monthly_postings
merge_df.head()

In [ ]:
df_not_remote_jobs = df[df['job_work_from_home']==False].copy()
# df_not_remote_jobs.head(3)
df_not_remote_salary = df_not_remote_jobs.pivot_table(values='salary_year_avg',index='job_title_short',aggfunc='mean').reset_index()
df_not_remote_salary.rename(columns={'salary_year_avg':'salary_not_remote_avg'},inplace=True)
# df_not_remote_salary
merge_df = pd.merge(df_not_remote_jobs,df_not_remote_salary,how='right',on='job_title_short')
merge_df.head()

In [ ]:
df_not_remote_jobs = df[df['job_work_from_home']==False].copy()
# df_not_remote_jobs.head(3)
df_not_remote_salary = df_not_remote_jobs.groupby('job_title_short')['salary_year_avg'].mean().reset_index()
# df_not_remote_salary
df_not_remote_salary.rename(columns={'salary_year_avg':'salary_not_remote_avg'},inplace=True)
merge_df = pd.merge(df_not_remote_jobs,df_not_remote_salary,how='right',on='job_title_short')
merge_df.head()

In [ ]:
df_company_salary = df.groupby('company_name')['salary_year_avg'].mean().reset_index()
# df_company_salary
df_company_remote = df[df['job_work_from_home']==True].groupby('company_name').size().reset_index(name='remote_count')
# df_company_remote
merge_df = pd.merge(df_company_salary, df_company_remote, how='inner',on='company_name')
merge_df.head()

In [ ]:
df['job_posted_month'] = df['job_posted_date'].dt.strftime('%b')
df

In [ ]:
months = df['job_posted_month'].unique()
print(months)

# write this with dict comprehension
dict_months = {month: df[df['job_posted_month'] == month] for month in months}
# dict_months


# long way
# dict_months = {}
# for month in months:
#     dict_months[month] = df[df['job_posted_month'] == month]

type(dict_months)
type(dict_months['Jun'])

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills,job_posted_month
